In [3]:
import pandas as pd

In [4]:
excel_path = "mouse_data.xlsx"
xls = pd.ExcelFile(excel_path)
act_df = pd.read_excel(xls, sheet_name = "Fem Act")
temp_df = pd.read_excel(xls, sheet_name = "Fem Temp")

In [5]:
MINUTES_PER_DAY = 1440

for df in (act_df, temp_df):
    df["day"] = df.index // MINUTES_PER_DAY
    df["minute"] = df.index % MINUTES_PER_DAY
    df["estrus"] = (df["day"] % 4 == 2)

In [6]:
mouse_cols = [
    c for c in act_df.columns if c not in ("day", "minute", "estrus")
]

activity_long = act_df.melt(
    id_vars = ["day", "minute", "estrus"],
    value_vars = mouse_cols,
    var_name = "mouse",
    value_name = "activity"
)

temperature_long = temp_df.melt(
    id_vars = ["day", "minute", "estrus"],
    value_vars = mouse_cols,
    var_name = "mouse",
    value_name = "temperature"
)

In [7]:
merged = pd.merge(
    activity_long,
    temperature_long,
    on = ["day", "minute", "estrus", "mouse"]
)

BIN_SIZE = 60
merged["hour"] = merged["minute"] // BIN_SIZE

hourly = (
    merged
    .groupby(["day", "hour", "estrus"])
    .agg(
        activity_mean = ("activity", "mean"),
        temperature_mean = ("temperature", "mean")
    )
    .reset_index()
)

In [8]:
hourly.to_json("../data/hourly_phase1.json", orient = "records", indent = 2)